In [1]:
%%capture
!pip install ipywidgets

In [2]:
%%capture
! pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu116
! pip install git+https://github.com/keepsimpler/sunyata
! pip install pytorch-lightning
! pip install lightning-bolts
! pip install einops

In [3]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

from sunyata.pytorch.data.tiny_imagenet import TinyImageNet, TinyImageNetDataModule

import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.loggers import CSVLogger

from sunyata.pytorch.arch.base import BaseModule, Residual
from sunyata.pytorch.arch.foldnet import FoldNet, FoldNetCfg, FoldNetRepeat, FoldNetRepeat2, Block

In [30]:
cfg = FoldNetCfg(
    block = Block,
    hidden_dim = 512+128,
    num_layers = 60,
    kernel_size = 5,
    patch_size = 2,
    num_classes = 200,
    fold_num = 1,
    drop_rate = 0.1,
    
    batch_size = 128,
    num_epochs = 100,
    learning_rate = 1e-2,
    optimizer_method = "AdamW",
    weight_decay = 0.1,
    learning_rate_scheduler= "LinearWarmupCosineAnnealingLR",
    warmup_epochs = 10,  # 2//5 * num_epoches
    warmup_start_lr = 1e-5,
)

In [5]:
normalize = torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

train_transforms = torchvision.transforms.Compose([
    # torchvision.transforms.RandomResizedCrop(64, scale=(1.0, 1.0), ratio=(1.0, 1.0)),
    torchvision.transforms.RandomHorizontalFlip(p=0.5),
    torchvision.transforms.RandAugment(num_ops=2, magnitude=12),
    torchvision.transforms.ColorJitter(0, 0, 0),
    torchvision.transforms.ToTensor(),
    normalize,
    # torchvision.transforms.RandomErasing(p=0)
])

val_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToTensor(),
        normalize,
    ]
)



tiny_image_net_datamodule = TinyImageNetDataModule(
    batch_size=cfg.batch_size, root='/datasets/',
    num_workers = 8, pin_memory=True,
    train_transforms=train_transforms, 
    val_transforms=val_transforms)



can not write to csv file.
can not write to csv file.


In [31]:
# input, target = next(iter(tiny_image_net_datamodule.train_dataloader()))
# input.shape, target.shape

In [32]:
model = FoldNetRepeat(cfg)

pl.utilities.model_summary.summarize(model,max_depth=1)

  | Name   | Type       | Params
--------------------------------------
0 | layers | ModuleList | 25.8 M
1 | embed  | Sequential | 9.6 K 
2 | digup  | Sequential | 128 K 
--------------------------------------
25.9 M    Trainable params
0         Non-trainable params
25.9 M    Total params
103.617   Total estimated model params size (MB)

In [8]:
trainer = pl.Trainer(
    # progress_bar_refresh_rate=10,
    precision=16,
    max_epochs=cfg.num_epochs,
    accelerator='gpu',
    devices=1,
    enable_checkpointing=True,
    # detect_anomaly=True,
    logger=CSVLogger("lightning_logs/", name="convmixer"),
    callbacks=[LearningRateMonitor(logging_interval="epoch")],
)    


Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [9]:
# trainer.fit(model, tiny_image_net_datamodule)

In [10]:
trainer.fit(model, 
            datamodule = tiny_image_net_datamodule,
            ckpt_path = "/notebooks/lightning_logs/convmixer/version_350/checkpoints/epoch=90-step=47702.ckpt",
           )


Restoring states from the checkpoint path at /notebooks/lightning_logs/convmixer/version_350/checkpoints/epoch=90-step=47702.ckpt


can not write to csv file.
can not write to csv file.


/usr/local/lib/python3.9/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:345: UserWarning: The dirpath has changed from 'lightning_logs/convmixer/version_350/checkpoints' to 'lightning_logs/convmixer/version_351/checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `best_model_path` will be reloaded.
  warnings.warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type       | Params
--------------------------------------
0 | layers | ModuleList | 19.6 M
1 | embed  | Sequential | 11.5 K
2 | digup  | Sequential | 461 K 
--------------------------------------
20.1 M    Trainable params
0         Non-trainable params
20.1 M    Total params
40.217    Total estimated model params size (MB)
Restored all states from the checkpoint file at /notebooks/lightning_logs/convmixer/version_350/checkpoints/epoch=90-step=47702.ckpt


Sanity Checking: 0it [00:00, ?it/s]

Training: 782it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
